In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
import re
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_regression, VarianceThreshold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import randint, uniform, boxcox
import xgboost as XGBRegressor
from sklearn.inspection import permutation_importance, partial_dependence
import warnings
import joblib
from scipy import stats
import time

# Start timer
start_time = time.time()

# Suppress warnings
warnings.filterwarnings('ignore')

# Set up matplotlib style for professional visualization
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
colors = sns.color_palette("viridis", 8)

# Create output directory
output_dir = "Predictions-Final"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)  # Remove if exists
os.makedirs(output_dir, exist_ok=True)

# Create subdirectories
os.makedirs(os.path.join(output_dir, "figures"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "data"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "models"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "results"), exist_ok=True)

# File path
file_path = r"C:\Users\Dani\AXF-1 data update 29APR25.xlsx"

def save_figure(fig, filename, dpi=300):
    """Safely save a figure with proper path handling"""
    # Replace problematic characters in filename
    safe_filename = re.sub(r'[\\/*?:"<>|]', "_", filename)
    # Save to the output directory
    fig_path = os.path.join(output_dir, "figures", safe_filename)
    fig.savefig(fig_path, dpi=dpi, bbox_inches='tight')
    plt.close(fig)
    return fig_path

def load_and_explore_data(file_path):
    """Load and explore the dataset"""
    print("Loading data...")
    df = pd.read_excel(file_path)
    
    print(f"\nDataset shape: {df.shape}")
    print("\nFirst 5 rows:")
    print(df.head())
    
    print("\nColumn information:")
    print(df.info())
    
    print("\nSummary statistics:")
    print(df.describe())
    
    # Check for missing values
    missing_values = df.isnull().sum()
    missing_percentage = (missing_values / len(df)) * 100
    
    print("\nMissing values by column:")
    missing_df = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': missing_percentage
    })
    print(missing_df[missing_df['Missing Values'] > 0].sort_values('Percentage', ascending=False))
    
    # Save missing values table
    missing_df.to_csv(os.path.join(output_dir, 'data', 'missing_values.csv'))
    
    # Check target variable
    target = '% dry after 24h in incubator'
    if target in df.columns:
        print(f"\nTarget variable '{target}' statistics:")
        print(df[target].describe())
        
        # Distribution plot for target
        plt.figure(figsize=(12, 8))
        ax = sns.histplot(df[target].dropna(), kde=True, bins=20, color=colors[0])
        ax.set_title(f'Distribution of {target}', fontsize=15)
        ax.set_xlabel(target, fontsize=12)
        ax.set_ylabel('Frequency', fontsize=12)
        plt.tight_layout()
        save_figure(plt.gcf(), 'target_distribution.png')
        
        # Check for skewness
        skewness = df[target].skew()
        print(f"Target skewness: {skewness}")
        
        if abs(skewness) > 1:
            print("Target is highly skewed, consider transformation")
            # Try log transformation
            plt.figure(figsize=(12, 8))
            ax = sns.histplot(np.log1p(df[target].dropna()), kde=True, bins=20, color=colors[1])
            ax.set_title(f'Log-transformed Distribution of {target}', fontsize=15)
            ax.set_xlabel(f'Log(1+{target})', fontsize=12)
            ax.set_ylabel('Frequency', fontsize=12)
            plt.tight_layout()
            save_figure(plt.gcf(), 'target_log_distribution.png')
    else:
        print(f"\nTarget variable '{target}' not found in columns.")
        # Look for similar column names
        similar_cols = [col for col in df.columns if 'dry' in col.lower() or '24h' in col.lower()]
        if similar_cols:
            print(f"Found similar columns: {similar_cols}")
            target = similar_cols[0]  # Use the first similar column as target
            print(f"Using '{target}' as target variable")
    
    return df, target

def identify_relevant_columns(df, target):
    """Identify columns that are relevant for prediction and exclude metadata and outcome columns"""
    # Define outcome columns that should not be used for prediction
    outcome_columns = [
        '% dry after 24h in incubator',        # Target variable
        '% dry after 2 days 70% humidity',      # Related outcome 
        'Droplet/particle size (µm)',           # Outcome measurement
        'Droplet/particle size range',          # Outcome measurement
        'Droplet/particle size range STDEV',    # Outcome measurement
        '% single core',                        # Outcome measurement
        '% empty',                              # Outcome measurement
        'Polydispersity Index',                 # Outcome measurement
        'Transmission window'                   # Outcome measurement
    ]
    
    # Columns to exclude (metadata, identifiers, experimental notes)
    metadata_columns = [
        'Column 1',                # Sample ID
        'Date of experiment',      # Timestamp
        'Aims & Hypothesis',       # Experimental context
        'Notes'                    # Free text notes
    ]
    
    # Remove UV viscosity (cP) due to multicollinearity with Emulsion viscosity
    multicollinearity_columns = [
        'UV viscosity (cP)'
    ]
    
    # Columns with too many missing values
    high_missing_columns = [
        'Temperature of Outer Phase'  # >50% missing values
    ]
    
    # Get all columns
    all_cols = df.columns.tolist()
    
    # Remove excluded columns
    excluded_columns = outcome_columns + metadata_columns + multicollinearity_columns + high_missing_columns
    relevant_cols = [col for col in all_cols if col not in excluded_columns]
    
    print(f"\nExcluding metadata columns: {metadata_columns}")
    print(f"Excluding outcome columns: {outcome_columns}")
    print(f"Excluding multicollinear columns: {multicollinearity_columns}")
    print(f"Excluding high missing value columns: {high_missing_columns}")
    print(f"Using {len(relevant_cols)} columns for analysis:")
    print(relevant_cols)
    
    # Save the list of relevant columns
    with open(os.path.join(output_dir, 'data', 'relevant_columns.txt'), 'w') as f:
        f.write('\n'.join(relevant_cols))
    
    return relevant_cols

def analyze_missing_values(df, relevant_cols):
    """Analyze missing value patterns to inform imputation strategy"""
    print("\nAnalyzing missing value patterns...")
    
    # Missing value correlation
    missing_matrix = df[relevant_cols].isnull().corr()
    
    # Plot heatmap of missing value correlation
    plt.figure(figsize=(16, 14))
    mask = np.triu(np.ones_like(missing_matrix))
    ax = sns.heatmap(missing_matrix, mask=mask, annot=False, cmap='coolwarm', 
                    vmin=-1, vmax=1, center=0)
    ax.set_title('Missing Value Correlation Matrix', fontsize=16)
    plt.tight_layout()
    save_figure(plt.gcf(), 'missing_value_correlation.png')
    
    # Group columns by missing percentage
    missing_percentage = df[relevant_cols].isnull().mean() * 100
    
    # Save missing percentage for relevant columns
    missing_percentage.to_csv(os.path.join(output_dir, 'data', 'relevant_columns_missing_percentage.csv'))
    
    # Bar plot of missing percentages
    plt.figure(figsize=(14, 10))
    missing_sorted = missing_percentage.sort_values(ascending=False)
    ax = sns.barplot(x=missing_sorted.index, y=missing_sorted.values, palette='viridis')
    ax.set_title('Missing Value Percentages by Column', fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.set_ylabel('Missing Percentage (%)', fontsize=14)
    ax.axhline(y=50, color='red', linestyle='--', label='50% threshold')
    ax.axhline(y=30, color='orange', linestyle='--', label='30% threshold')
    ax.axhline(y=10, color='green', linestyle='--', label='10% threshold')
    ax.legend()
    plt.tight_layout()
    save_figure(plt.gcf(), 'missing_percentages.png')
    
    # Categories of missing data
    low_missing = missing_percentage[missing_percentage < 10].index.tolist()
    medium_missing = missing_percentage[(missing_percentage >= 10) & (missing_percentage < 30)].index.tolist()
    high_missing = missing_percentage[(missing_percentage >= 30) & (missing_percentage < 50)].index.tolist()
    very_high_missing = missing_percentage[missing_percentage >= 50].index.tolist()
    
    print(f"\nColumns with low missing percentage (<10%): {len(low_missing)}")
    for col in low_missing:
        print(f"  - {col}: {missing_percentage[col]:.2f}%")
    
    print(f"\nColumns with medium missing percentage (10-30%): {len(medium_missing)}")
    for col in medium_missing:
        print(f"  - {col}: {missing_percentage[col]:.2f}%")
    
    print(f"\nColumns with high missing percentage (30-50%): {len(high_missing)}")
    for col in high_missing:
        print(f"  - {col}: {missing_percentage[col]:.2f}%")
    
    # Check if any columns should be dropped due to too many missing values
    if very_high_missing:
        print(f"\nColumns with >50% missing values (consider dropping): {len(very_high_missing)}")
        for col in very_high_missing:
            print(f"  - {col}: {missing_percentage[col]:.2f}%")
    
    # Save missing value analysis to file
    with open(os.path.join(output_dir, 'results', 'missing_value_analysis.txt'), 'w') as f:
        f.write("Missing Value Analysis\n")
        f.write("=====================\n\n")
        
        f.write(f"Columns with low missing percentage (<10%): {len(low_missing)}\n")
        for col in low_missing:
            f.write(f"  - {col}: {missing_percentage[col]:.2f}%\n")
        
        f.write(f"\nColumns with medium missing percentage (10-30%): {len(medium_missing)}\n")
        for col in medium_missing:
            f.write(f"  - {col}: {missing_percentage[col]:.2f}%\n")
        
        f.write(f"\nColumns with high missing percentage (30-50%): {len(high_missing)}\n")
        for col in high_missing:
            f.write(f"  - {col}: {missing_percentage[col]:.2f}%\n")
        
        if very_high_missing:
            f.write(f"\nColumns with >50% missing values (consider dropping): {len(very_high_missing)}\n")
            for col in very_high_missing:
                f.write(f"  - {col}: {missing_percentage[col]:.2f}%\n")
    
    return {
        'low_missing': low_missing,
        'medium_missing': medium_missing,
        'high_missing': high_missing,
        'very_high_missing': very_high_missing
    }

def analyze_categorical_features(df, target, relevant_cols):
    """Analyze categorical features and their relationship with target"""
    print("\nAnalyzing categorical features...")
    
    # Identify categorical columns from relevant columns
    categorical_cols = df[relevant_cols].select_dtypes(include=['object']).columns.tolist()
    print(f"Relevant categorical columns: {categorical_cols}")
    
    # Create summary dataframe for categorical columns
    cat_summary = []
    
    # For each categorical column, check distribution and relationship with target
    for col in categorical_cols:
        print(f"\nAnalyzing {col}:")
        # Distribution
        value_counts = df[col].value_counts()
        unique_count = len(value_counts)
        top_value = value_counts.index[0]
        top_value_percent = (value_counts.iloc[0] / value_counts.sum()) * 100
        
        print(f"Number of unique values: {unique_count}")
        print("Top 5 most common values:")
        print(value_counts.head())
        
        # Save value counts
        value_counts.to_csv(os.path.join(output_dir, 'data', f'{col}_value_counts.csv'))
        
        # Add to summary
        cat_summary.append({
            'Column': col,
            'Unique_Values': unique_count,
            'Top_Value': top_value,
            'Top_Value_Percent': top_value_percent,
            'Missing_Percent': df[col].isnull().mean() * 100
        })
        
        # Check if too many unique values (potential issue for one-hot encoding)
        if unique_count > 10:
            print(f"WARNING: Column {col} has {unique_count} unique values. May cause overfitting with one-hot encoding.")
        
        # Plot value distribution
        plt.figure(figsize=(14, 8))
        ax = value_counts.nlargest(10).plot(kind='bar', color=colors[2])
        ax.set_title(f'Top 10 values in {col}', fontsize=15)
        ax.set_xlabel(col, fontsize=12)
        ax.set_ylabel('Count', fontsize=12)
        plt.tight_layout()
        save_figure(plt.gcf(), f'{col}_distribution.png')
        
        # Box plot of target by categorical variable (top 5 categories)
        if target in df.columns:
            plt.figure(figsize=(16, 10))
            top_categories = value_counts.nlargest(5).index
            filtered_data = df[df[col].isin(top_categories)]
            
            if not filtered_data.empty and not filtered_data[target].dropna().empty:
                ax = sns.boxplot(x=col, y=target, data=filtered_data, palette='viridis')
                ax.set_title(f'Relationship between {col} and {target}', fontsize=15)
                ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
                ax.set_xlabel(col, fontsize=12)
                ax.set_ylabel(target, fontsize=12)
                plt.tight_layout()
                save_figure(plt.gcf(), f'{col}_target_relationship.png')
                
                # Calculate ANOVA to test if categories are significantly different
                categories = filtered_data[col].unique()
                if len(categories) > 1:
                    anova_data = []
                    for category in categories:
                        cat_values = filtered_data[filtered_data[col] == category][target].dropna()
                        if len(cat_values) > 0:
                            anova_data.append(cat_values)
                    
                    if len(anova_data) > 1 and all(len(x) > 0 for x in anova_data):
                        try:
                            f_stat, p_value = stats.f_oneway(*anova_data)
                            print(f"ANOVA for {col}: F-statistic = {f_stat:.4f}, p-value = {p_value:.4f}")
                            
                            # Add to summary
                            cat_summary[-1]['ANOVA_p_value'] = p_value
                            cat_summary[-1]['Significant'] = p_value < 0.05
                        except:
                            print(f"Could not perform ANOVA for {col}")
    
    # Save categorical summary
    pd.DataFrame(cat_summary).to_csv(os.path.join(output_dir, 'results', 'categorical_summary.csv'), index=False)
    
    return categorical_cols

def analyze_numerical_features(df, target, relevant_cols):
    """Analyze numerical features and their relationship with target"""
    print("\nAnalyzing numerical features...")
    
    # Identify numerical columns from relevant columns
    numerical_cols = df[relevant_cols].select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    print(f"Relevant numerical columns: {numerical_cols}")
    
    # Create summary dataframe for numerical columns
    num_summary = []
    
    # Analyze distributions
    for col in numerical_cols:
        # Get statistics
        stats_dict = {
            'Column': col,
            'Mean': df[col].mean(),
            'Median': df[col].median(),
            'Std': df[col].std(),
            'Min': df[col].min(),
            'Max': df[col].max(),
            'Missing_Percent': df[col].isnull().mean() * 100,
            'Skewness': df[col].skew(),
            'Kurtosis': df[col].kurtosis()
        }
        
        # Plot distribution and qq-plot
        plt.figure(figsize=(15, 6))
        
        # Histogram with KDE
        plt.subplot(1, 2, 1)
        sns.histplot(df[col].dropna(), kde=True, color=colors[3])
        plt.title(f'Distribution of {col}')
        plt.xlabel(col)
        
        # QQ plot
        plt.subplot(1, 2, 2)
        stats.probplot(df[col].dropna(), dist="norm", plot=plt)
        plt.title(f'Q-Q Plot of {col} (Skewness: {stats_dict["Skewness"]:.2f})')
        
        plt.tight_layout()
        save_figure(plt.gcf(), f'{col}_distribution_qq.png')
        
        num_summary.append(stats_dict)
    
    # Save numerical summary
    pd.DataFrame(num_summary).to_csv(os.path.join(output_dir, 'results', 'numerical_summary.csv'), index=False)
    
    # Compute correlation with target
    if target in df.columns:
        # Filter for rows where target is not null
        df_corr = df[numerical_cols + [target]].dropna(subset=[target])
        correlations = df_corr.corr()[target].sort_values(ascending=False)
        print("\nCorrelations with target variable:")
        print(correlations)
        
        # Save correlations
        correlations.to_csv(os.path.join(output_dir, 'results', 'target_correlations.csv'))
        
        # Plot correlation heatmap
        plt.figure(figsize=(14, 12))
        correlation_matrix = df_corr.corr()
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        ax = sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
                    square=True, linewidths=.5)
        ax.set_title('Correlation Matrix', fontsize=16)
        plt.tight_layout()
        save_figure(plt.gcf(), 'correlation_heatmap.png')
        
        # Plot correlations with target as bar chart
        plt.figure(figsize=(14, 10))
        correlations_without_target = correlations.drop(target)
        ax = correlations_without_target.sort_values().plot(kind='barh', color=[colors[i % len(colors)] for i in range(len(correlations_without_target))])
        ax.set_title(f'Correlation with {target}', fontsize=15)
        ax.set_xlabel('Correlation Coefficient', fontsize=12)
        plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.tight_layout()
        save_figure(plt.gcf(), 'target_correlation_barchart.png')
        
        # Plot scatter plots for top correlated features
        top_correlated = correlations.drop(target).abs().nlargest(5).index.tolist()
        for col in top_correlated:
            plt.figure(figsize=(12, 8))
            ax = sns.scatterplot(x=col, y=target, data=df, alpha=0.6, color=colors[4])
            ax.set_title(f'Scatter plot of {target} vs {col}', fontsize=15)
            ax.set_xlabel(col, fontsize=12)
            ax.set_ylabel(target, fontsize=12)
            plt.tight_layout()
            save_figure(plt.gcf(), f'{col}_scatter.png')
            
            # Add regression plot
            plt.figure(figsize=(12, 8))
            ax = sns.regplot(x=col, y=target, data=df, scatter_kws={'alpha':0.5}, line_kws={'color':colors[5]})
            ax.set_title(f'Regression plot of {target} vs {col}', fontsize=15)
            ax.set_xlabel(col, fontsize=12)
            ax.set_ylabel(target, fontsize=12)
            
            # Add correlation coefficient in the plot
            corr_val = df[[col, target]].corr().iloc[0, 1]
            ax.annotate(f'r = {corr_val:.3f}', xy=(0.05, 0.95), xycoords='axes fraction',
                       fontsize=12, bbox=dict(boxstyle="round,pad=0.3", fc="white", alpha=0.8))
            
            plt.tight_layout()
            save_figure(plt.gcf(), f'{col}_regplot.png')
    
    # Check for collinearity among numerical features
    correlation_matrix_num = df[numerical_cols].corr()
    
    # Find highly correlated features
    high_corr_threshold = 0.8
    high_corr_pairs = []
    
    for i in range(len(correlation_matrix_num.columns)):
        for j in range(i+1, len(correlation_matrix_num.columns)):
            if abs(correlation_matrix_num.iloc[i, j]) > high_corr_threshold:
                high_corr_pairs.append(
                    (correlation_matrix_num.columns[i], 
                     correlation_matrix_num.columns[j], 
                     correlation_matrix_num.iloc[i, j])
                )
    
    if high_corr_pairs:
        print("\nHighly correlated feature pairs (correlation > 0.8):")
        for col1, col2, corr in high_corr_pairs:
            print(f"{col1} and {col2}: {corr:.3f}")
            
        # Save highly correlated pairs
        pd.DataFrame(high_corr_pairs, columns=['Feature 1', 'Feature 2', 'Correlation']) \
            .to_csv(os.path.join(output_dir, 'results', 'high_correlation_pairs.csv'), index=False)
        
        # Plot correlation network for highly correlated pairs
        if len(high_corr_pairs) > 0:
            try:
                import networkx as nx
                
                plt.figure(figsize=(12, 10))
                G = nx.Graph()
                
                # Add nodes
                for col in numerical_cols:
                    G.add_node(col)
                
                # Add edges with weights
                for col1, col2, corr in high_corr_pairs:
                    G.add_edge(col1, col2, weight=abs(corr))
                
                # Set positions
                pos = nx.spring_layout(G, k=0.5, iterations=50)
                
                # Draw
                nx.draw_networkx_nodes(G, pos, node_size=700, node_color=colors[6], alpha=0.8)
                nx.draw_networkx_labels(G, pos, font_size=10)
                
                # Draw edges with width proportional to correlation
                for u, v, d in G.edges(data=True):
                    width = d['weight'] * 5
                    nx.draw_networkx_edges(G, pos, edgelist=[(u, v)], width=width, alpha=0.7)
                
                # Add edge labels (correlations)
                edge_labels = nx.get_edge_attributes(G, 'weight')
                edge_labels = {k: f'{v:.2f}' for k, v in edge_labels.items()}
                nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
                
                plt.title('Correlation Network (correlations > 0.8)', fontsize=15)
                plt.axis('off')
                plt.tight_layout()
                save_figure(plt.gcf(), 'correlation_network.png')
                
            except ImportError:
                print("NetworkX library not available. Skipping correlation network plot.")
    
    return numerical_cols

def filter_categorical_columns(df, categorical_cols, threshold=10):
    """Filter categorical columns based on cardinality to avoid overfitting"""
    filtered_categorical_cols = []
    high_cardinality_cols = []
    
    for col in categorical_cols:
        n_unique = df[col].nunique()
        if n_unique <= threshold:
            filtered_categorical_cols.append(col)
        else:
            high_cardinality_cols.append(col)
    
    if high_cardinality_cols:
        print(f"\nExcluding high-cardinality categorical columns to avoid overfitting: {high_cardinality_cols}")
    
    print(f"Using {len(filtered_categorical_cols)} categorical columns for modeling")
    
    # Save the list of filtered categorical columns
    with open(os.path.join(output_dir, 'data', 'filtered_categorical_columns.txt'), 'w') as f:
        f.write('\n'.join(filtered_categorical_cols))
    
    return filtered_categorical_cols

def feature_engineering(df, numerical_cols, categorical_cols, target):
    """Create new features using various methods with physical meaning"""
    print("\nPerforming feature engineering...")
    
    # Define outcome columns that should not be used for prediction
    outcome_columns = [
        '% dry after 24h in incubator',        # Target variable
        '% dry after 2 days 70% humidity',      # Related outcome 
        'Droplet/particle size (µm)',           # Outcome measurement
        'Droplet/particle size range',          # Outcome measurement
        'Droplet/particle size range STDEV',    # Outcome measurement
        '% single core',                        # Outcome measurement
        '% empty',                              # Outcome measurement
        'Polydispersity Index',                 # Outcome measurement
        'Transmission window'                   # Outcome measurement
    ]
    
    df_engineered = df.copy()
    created_features = []
    
    # 1. INTERACTION FEATURES
    # Identify numerical columns with highest correlation to target
    if target in df.columns:
        target_data = df_engineered[[target] + numerical_cols].dropna()
        correlations = target_data.corr()[target].abs().sort_values(ascending=False)
        top_features = correlations.drop(target).head(5).index.tolist()
        
        print(f"\nCreating interaction features for top correlated features: {top_features}")
        
        # Create interactions between top features
        for i in range(len(top_features)):
            for j in range(i + 1, len(top_features)):
                col1 = top_features[i]
                col2 = top_features[j]
                
                # Multiplication interaction
                interaction_name = f"{col1}_x_{col2}"
                df_engineered[interaction_name] = df_engineered[col1] * df_engineered[col2]
                created_features.append({'name': interaction_name, 'type': 'interaction', 'formula': f"{col1} * {col2}"})
                
                # Ratio interaction (if it makes physical sense)
                ratio_name = f"{col1}_to_{col2}"
                # Avoid division by zero
                df_engineered[ratio_name] = df_engineered[col1] / df_engineered[col2].replace(0, np.nan)
                created_features.append({'name': ratio_name, 'type': 'ratio', 'formula': f"{col1} / {col2}"})
        
        # Create physically meaningful combinations
        # Flow-related ratios
        if 'Dispersed Flow Rate (mL/min)' in numerical_cols and 'Continuous Flow Rate (mL/min)' in numerical_cols:
            # Flow ratio
            df_engineered['Flow_Ratio'] = df_engineered['Dispersed Flow Rate (mL/min)'] / df_engineered['Continuous Flow Rate (mL/min)'].replace(0, np.nan)
            created_features.append({'name': 'Flow_Ratio', 'type': 'physical_ratio', 'formula': 'Dispersed Flow Rate / Continuous Flow Rate'})
            
        # Energy density
        if 'UV Power (J s-1)' in numerical_cols and 'Dispersed Flow Rate (mL/min)' in numerical_cols:
            # Energy per volume
            df_engineered['Energy_Density'] = df_engineered['UV Power (J s-1)'] / df_engineered['Dispersed Flow Rate (mL/min)'].replace(0, np.nan)
            created_features.append({'name': 'Energy_Density', 'type': 'physical_ratio', 'formula': 'UV Power / Dispersed Flow Rate'})
        
        # Residence time approximation
        if 'Length of curing tubing (m)' in numerical_cols and 'Continuous Flow Rate (mL/min)' in numerical_cols:
            # Residence time = length / flow rate
            df_engineered['Residence_Time'] = df_engineered['Length of curing tubing (m)'] / df_engineered['Continuous Flow Rate (mL/min)'].replace(0, np.nan)
            created_features.append({'name': 'Residence_Time', 'type': 'physical_ratio', 'formula': 'Length of curing tubing / Continuous Flow Rate'})
        
        # Viscosity-related features
        if 'Emulsion viscosity (cP)' in numerical_cols and 'Core Viscosity (cP)' in numerical_cols:
            # Viscosity ratio
            df_engineered['Viscosity_Ratio'] = df_engineered['Emulsion viscosity (cP)'] / df_engineered['Core Viscosity (cP)'].replace(0, np.nan)
            created_features.append({'name': 'Viscosity_Ratio', 'type': 'physical_ratio', 'formula': 'Emulsion viscosity / Core Viscosity'})
        
        print(f"Created {len(created_features)} interaction and physical features")
    
    # 2. POLYNOMIAL FEATURES
    # Create polynomial features for top numerical features
    poly_features = []
    if len(top_features) > 0:
        for feature in top_features[:3]:  # Limit to top 3 to avoid explosion of features
            # Square
            square_name = f"{feature}_squared"
            df_engineered[square_name] = df_engineered[feature] ** 2
            poly_features.append({'name': square_name, 'type': 'polynomial', 'formula': f"{feature}^2"})
            
            # Square root (for positive features)
            if df_engineered[feature].dropna().min() >= 0:
                sqrt_name = f"{feature}_sqrt"
                df_engineered[sqrt_name] = np.sqrt(df_engineered[feature])
                poly_features.append({'name': sqrt_name, 'type': 'polynomial', 'formula': f"sqrt({feature})"})
        
        created_features.extend(poly_features)
        print(f"Created {len(poly_features)} polynomial features")
    
    # 3. BINNING
    # Create bins for selected numerical features
    binned_features = []
    for feature in top_features[:2]:  # Limit to top 2
        binned_name = f"{feature}_binned"
        # Create 5 bins based on quantiles
        df_engineered[binned_name] = pd.qcut(df_engineered[feature], q=5, labels=False, duplicates='drop')
        binned_features.append({'name': binned_name, 'type': 'binned', 'formula': f"qcut({feature}, 5)"})
    
    created_features.extend(binned_features)
    print(f"Created {len(binned_features)} binned features")
    
    # 4. TRANSFORMATIONS
    # Apply transformations to skewed features
    transformed_features = []
    for col in numerical_cols:
        # Skip columns with negative values or all zeros for log transform
        if df_engineered[col].dropna().min() <= 0:
            continue
        
        # Check skewness
        skewness = df_engineered[col].skew()
        
        # Apply transformations to highly skewed features
        if abs(skewness) > 1:
            # Log transformation
            log_name = f"{col}_log"
            df_engineered[log_name] = np.log1p(df_engineered[col])
            transformed_features.append({'name': log_name, 'type': 'log_transform', 'formula': f"log(1+{col})"})
    
    created_features.extend(transformed_features)
    print(f"Created {len(transformed_features)} transformed features")
    
    # Save created features information
    pd.DataFrame(created_features).to_csv(os.path.join(output_dir, 'results', 'engineered_features.csv'), index=False)
    
    # Visualize distribution of top engineered features
    # Select a few top engineered features for visualization
    if len(created_features) > 0:
        top_engineered = [f['name'] for f in created_features[:min(6, len(created_features))]]
        for col in top_engineered:
            if col in df_engineered.columns:
                plt.figure(figsize=(12, 6))
                sns.histplot(df_engineered[col].dropna(), kde=True, color=colors[1])
                plt.title(f'Distribution of {col}')
                plt.tight_layout()
                save_figure(plt.gcf(), f'engineered_{col}_distribution.png')
    
    # Get new lists of feature columns
    new_numerical_cols = df_engineered.select_dtypes(include=['int64', 'float64']).columns.tolist()
    new_numerical_cols = [col for col in new_numerical_cols if col != target and col not in outcome_columns]
    
    return df_engineered, new_numerical_cols, created_features

def handle_outliers(df, numerical_cols, method='iqr', threshold=1.5):
    """Detect and handle outliers in numerical features"""
    print("\nHandling outliers in numerical features...")
    
    # Copy the original dataframe to avoid modifying it
    df_processed = df.copy()
    
    # For each numerical column, detect and handle outliers
    outlier_info = {}
    
    for col in numerical_cols:
        # Skip columns with too many missing values
        if df_processed[col].isnull().sum() / len(df_processed) > 0.5:
            print(f"Skipping {col} for outlier detection due to high missing percentage")
            continue
        
        # Detect outliers
        if method == 'iqr':
            Q1 = df_processed[col].quantile(0.25)
            Q3 = df_processed[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            
            # Get outliers
            outliers = df_processed[(df_processed[col] < lower_bound) | (df_processed[col] > upper_bound)][col]
            
            outlier_info[col] = {
                'method': 'IQR',
                'threshold': threshold,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'outlier_count': len(outliers),
                'outlier_percentage': len(outliers) / df_processed[col].count() * 100
            }
            
            # Cap the outliers
            print(f"Capping outliers in {col} using IQR method")
            print(f"  - Lower bound: {lower_bound:.2f}")
            print(f"  - Upper bound: {upper_bound:.2f}")
            print(f"  - Outliers found: {len(outliers)} ({len(outliers) / df_processed[col].count() * 100:.2f}%)")
            
            df_processed[col] = df_processed[col].clip(lower=lower_bound, upper=upper_bound)
        
        elif method == 'zscore':
            z_scores = stats.zscore(df_processed[col].dropna())
            abs_z_scores = np.abs(z_scores)
            outliers_mask = abs_z_scores > threshold
            outliers = df_processed[col].dropna().iloc[outliers_mask]
            
            outlier_info[col] = {
                'method': 'Z-score',
                'threshold': threshold,
                'outlier_count': len(outliers),
                'outlier_percentage': len(outliers) / df_processed[col].count() * 100
            }
            
            # Replace outliers with median
            if len(outliers) > 0:
                print(f"Replacing outliers in {col} using Z-score method")
                print(f"  - Z-score threshold: {threshold}")
                print(f"  - Outliers found: {len(outliers)} ({len(outliers) / df_processed[col].count() * 100:.2f}%)")
                
                # Create a series with outlier indices
                outlier_indices = df_processed[col].dropna().iloc[outliers_mask].index
                df_processed.loc[outlier_indices, col] = df_processed[col].median()
    
    # Save outlier information
    pd.DataFrame(outlier_info).T.to_csv(os.path.join(output_dir, 'results', 'outlier_handling_info.csv'))
    
    # Visualize before/after for top columns with most outliers
    top_outlier_cols = sorted(
        [(col, info['outlier_percentage']) for col, info in outlier_info.items()], 
        key=lambda x: x[1], 
        reverse=True
    )[:5]
    
    if top_outlier_cols:
        # Create before/after comparison plots
        for col, pct in top_outlier_cols:
            plt.figure(figsize=(15, 6))
            
            # Before (original data)
            plt.subplot(1, 2, 1)
            sns.histplot(df[col].dropna(), kde=True, color=colors[0])
            plt.title(f'Before Outlier Handling: {col}\n({pct:.1f}% outliers)')
            
            # After (processed data)
            plt.subplot(1, 2, 2)
            sns.histplot(df_processed[col].dropna(), kde=True, color=colors[1])
            plt.title(f'After Outlier Handling: {col}')
            
            plt.tight_layout()
            save_figure(plt.gcf(), f'outlier_handling_{col}.png')
    
    return df_processed, outlier_info

def create_imputation_strategy(df, numerical_cols, categorical_cols, missing_patterns):
    """Create a comprehensive imputation strategy with indicator variables"""
    print("\nDesigning imputation strategy...")
    
    # Strategies for numerical features
    numerical_imputers = {}
    
    # For columns with low missing values - use median
    for col in [c for c in numerical_cols if c in missing_patterns['low_missing']]:
        numerical_imputers[col] = ('median', SimpleImputer(strategy='median'))
    
    # For columns with medium missing values - use KNN imputation
    medium_missing_num = [c for c in numerical_cols if c in missing_patterns['medium_missing']]
    if medium_missing_num:
        print(f"Using KNN imputation for {len(medium_missing_num)} numerical columns with medium missing values")
        for col in medium_missing_num:
            numerical_imputers[col] = ('knn', KNNImputer(n_neighbors=5))
    
    # For columns with high missing - consider dropping or advanced methods
    high_missing_num = [c for c in numerical_cols if c in missing_patterns['high_missing']]
    if high_missing_num:
        print(f"Columns with high missing values (consider dropping or more advanced imputation): {high_missing_num}")
        # Use KNN with more neighbors for more robust estimation
        for col in high_missing_num:
            numerical_imputers[col] = ('knn', KNNImputer(n_neighbors=10))
    
    # For categorical columns - always use most frequent
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    
    # Create list of columns to add indicator variables for
    indicator_cols = []
    indicator_cols.extend(medium_missing_num)
    indicator_cols.extend(high_missing_num)
    for col in categorical_cols:
        if df[col].isnull().mean() > 0.05:  # Add indicator if > 5% missing
            indicator_cols.append(col)
    
    # Add columns with very high missing values but which we'll keep
    very_high_missing_keep = [c for c in numerical_cols if c in missing_patterns['very_high_missing']]
    indicator_cols.extend(very_high_missing_keep)
    
    print("\nImputation strategy:")
    print("- Simple median imputation for numerical columns with <10% missing values")
    print("- KNN imputation for numerical columns with 10-30% missing values")
    print("- KNN imputation with more neighbors for columns with >30% missing values")
    print("- Most frequent value imputation for categorical columns")
    print(f"- Missing value indicators will be created for {len(indicator_cols)} columns")
    
    # Save imputation strategy
    with open(os.path.join(output_dir, 'results', 'imputation_strategy.txt'), 'w') as f:
        f.write("Imputation Strategy:\n\n")
        f.write("Numerical columns with low missing (<10%):\n")
        f.write("- Method: Median imputation\n")
        f.write("- Columns: " + ', '.join([c for c in numerical_cols if c in missing_patterns['low_missing']]) + "\n\n")
        
        if medium_missing_num:
            f.write("Numerical columns with medium missing (10-30%):\n")
            f.write("- Method: KNN imputation (n_neighbors=5)\n")
            f.write("- Columns: " + ', '.join(medium_missing_num) + "\n\n")
        
        if high_missing_num:
            f.write("Numerical columns with high missing (30-50%):\n")
            f.write("- Method: KNN imputation (n_neighbors=10)\n")
            f.write("- Columns: " + ', '.join(high_missing_num) + "\n\n")
        
        if very_high_missing_keep:
            f.write("Numerical columns with very high missing (>50%) that will be kept:\n")
            f.write("- Method: KNN imputation (n_neighbors=10) + missing indicator\n")
            f.write("- Columns: " + ', '.join(very_high_missing_keep) + "\n\n")
        
        f.write("Categorical columns:\n")
        f.write("- Method: Most frequent value imputation\n")
        f.write("- Columns: " + ', '.join(categorical_cols) + "\n\n")
        
        f.write("Missing value indicators will be created for:\n")
        f.write("- Columns: " + ', '.join(indicator_cols))
    
    return numerical_imputers, categorical_imputer, indicator_cols

def prepare_data(df, target, categorical_cols, numerical_cols, numerical_imputers, 
                categorical_imputer, indicator_cols, outlier_handling=True):
    """Prepare data for modeling with comprehensive preprocessing"""
    print("\nPreparing data for modeling...")
    
    # Make a copy to avoid modifying the original dataframe
    df_processed = df.copy()
    
    # Separate features and target
    if target in df.columns:
        X = df_processed[categorical_cols + numerical_cols].copy()
        y = df_processed[target].copy()
    else:
        raise ValueError(f"Target column '{target}' not found in dataframe")
    
    # Handle missing values in target
    mask = ~y.isna()
    X = X[mask]
    y = y[mask]
    
    print(f"Data after removing rows with missing target: {X.shape}")
    
    # Handle outliers if requested
    if outlier_handling:
        X, outlier_info = handle_outliers(X, numerical_cols)
        # Save outlier information
        pd.DataFrame(outlier_info).T.to_csv(os.path.join(output_dir, 'results', 'outlier_handling.csv'))
    
    # Create missing value indicator variables
    for col in indicator_cols:
        if col in X.columns:
            indicator_name = f"{col}_missing"
            X[indicator_name] = X[col].isnull().astype(int)
    
    # Apply imputation for numerical columns
    for col in numerical_cols:
        if col in X.columns and X[col].isnull().sum() > 0:
            # Get imputation strategy
            if col in numerical_imputers:
                strategy, imputer = numerical_imputers[col]
                print(f"Imputing {col} using {strategy} strategy")
                # Create a DataFrame with just this column
                col_df = X[[col]]
                # Impute
                imputed_values = imputer.fit_transform(col_df)
                # Replace in original DataFrame
                X[col] = imputed_values
    
    # Apply imputation for categorical columns
    for col in categorical_cols:
        if col in X.columns and X[col].isnull().sum() > 0:
            print(f"Imputing {col} using most frequent strategy")
            # Create a DataFrame with just this column
            col_df = X[[col]].astype(str)  # Convert to string to handle non-string objects
            # Impute
            imputed_values = categorical_imputer.fit_transform(col_df)
            # Replace in original DataFrame
            X[col] = imputed_values
    
    # Check if any missing values remain
    missing_after = X.isnull().sum().sum()
    if missing_after > 0:
        print(f"WARNING: {missing_after} missing values remain after imputation")
        # List columns with remaining missing values
        missing_cols = X.columns[X.isnull().sum() > 0].tolist()
        print(f"Columns with remaining missing values: {missing_cols}")
        
        # Fill any remaining missing values with conservative strategies
        for col in missing_cols:
            if col in numerical_cols:
                X[col] = X[col].fillna(X[col].median())
            else:
                X[col] = X[col].fillna(X[col].mode()[0])
    
    # Update numerical columns list to include indicators
    numerical_cols_with_indicators = numerical_cols.copy()
    for col in indicator_cols:
        indicator_name = f"{col}_missing"
        if indicator_name in X.columns:
            numerical_cols_with_indicators.append(indicator_name)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Training set size: {X_train.shape}")
    print(f"Testing set size: {X_test.shape}")
    
    # Save the training and testing sets
    X_train.to_csv(os.path.join(output_dir, 'data', 'X_train.csv'), index=False)
    X_test.to_csv(os.path.join(output_dir, 'data', 'X_test.csv'), index=False)
    y_train.to_csv(os.path.join(output_dir, 'data', 'y_train.csv'), index=False)
    y_test.to_csv(os.path.join(output_dir, 'data', 'y_test.csv'), index=False)
    
    return X_train, X_test, y_train, y_test, numerical_cols_with_indicators

def build_preprocessing_pipeline(numerical_cols, categorical_cols, scaling_method='standard'):
    """Build comprehensive preprocessing pipeline with multiple options"""
    print(f"\nBuilding preprocessing pipeline with {scaling_method} scaling...")
    
    transformers = []
    
    # For numerical features: choose scaling method
    if numerical_cols:
        if scaling_method == 'standard':
            # Standardization (z-score normalization)
            transformers.append(('num', StandardScaler(), numerical_cols))
        elif scaling_method == 'minmax':
            # Min-Max scaling (normalization to [0,1])
            transformers.append(('num', MinMaxScaler(), numerical_cols))
        elif scaling_method == 'robust':
            # Robust scaling (uses quantiles, less affected by outliers)
            transformers.append(('num', RobustScaler(), numerical_cols))
    
    # For categorical features: one-hot encoding
    if categorical_cols:
        transformers.append(('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols))
    
    # Combine transformers
    preprocessor = ColumnTransformer(transformers=transformers)
    
    return preprocessor

def select_features(X_train, y_train, numerical_cols, categorical_cols, preprocessor, 
                   method='importance', n_features=None):
    """Select the most important features using various methods"""
    print(f"\nPerforming feature selection using {method} method...")
    
    # Create a pipeline with preprocessing
    pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])
    
    # Transform the data
    X_train_processed = pipeline.fit_transform(X_train)
    
    # Get feature names after preprocessing
    feature_names = []
    if hasattr(preprocessor, 'transformers_'):
        for name, transformer, cols in preprocessor.transformers_:
            if name == 'num':
                feature_names.extend(cols)
            elif name == 'cat':
                # For categorical features, get one-hot encoded feature names
                ohe = transformer
                if hasattr(ohe, 'get_feature_names_out'):
                    cat_features = ohe.get_feature_names_out(cols)
                    feature_names.extend(cat_features)
                else:
                    # For categorical features without get_feature_names_out, approximate
                    for col in cols:
                        unique_values = X_train[col].dropna().unique()
                        for val in unique_values:
                            feature_names.append(f"{col}_{val}")
    
    # Adjust feature names and processed data to match in length
    min_len = min(len(feature_names), X_train_processed.shape[1])
    feature_names = feature_names[:min_len]
    X_train_processed = X_train_processed[:, :min_len]
    
    if method == 'importance':
        # Use Random Forest for feature importance
        if n_features is None:
            n_features = min(30, X_train_processed.shape[1])
        
        selector = SelectFromModel(
            RandomForestRegressor(n_estimators=100, random_state=42),
            max_features=n_features
        )
        selector.fit(X_train_processed, y_train)
        
        # Get selected feature indices
        selected_indices = selector.get_support()
        
        # Calculate importances
        importances = selector.estimator_.feature_importances_
        
    elif method == 'correlation':
        # Use SelectKBest with f_regression for correlation-based selection
        if n_features is None:
            n_features = min(20, X_train_processed.shape[1])
        
        selector = SelectKBest(f_regression, k=n_features)
        selector.fit(X_train_processed, y_train)
        
        # Get selected feature indices
        selected_indices = selector.get_support()
        
        # Calculate importances
        importances = selector.scores_
        
    elif method == 'stability':
        # Stability selection - run multiple feature selections with subsampling
        if n_features is None:
            n_features = min(30, X_train_processed.shape[1])
        
        # Initialize counters for feature selection frequency
        selection_counts = np.zeros(X_train_processed.shape[1])
        n_iterations = 20
        
        base_selector = SelectFromModel(
            RandomForestRegressor(n_estimators=100, random_state=42),
            max_features=n_features
        )
        
        for i in range(n_iterations):
            # Create a random subsample
            indices = np.random.choice(
                np.arange(X_train_processed.shape[0]),
                size=int(0.8 * X_train_processed.shape[0]),
                replace=False
            )
            X_subsample = X_train_processed[indices]
            y_subsample = y_train.iloc[indices]
            
            # Fit on subsample
            base_selector.fit(X_subsample, y_subsample)
            
            # Add to selection counts
            selection_counts += base_selector.get_support().astype(int)
        
        # Select features that were selected in at least 50% of iterations
        selected_indices = selection_counts >= (n_iterations / 2)
        
        # Calculate importances as the fraction of times each feature was selected
        importances = selection_counts / n_iterations
    
    # Get selected feature names
    selected_features = [feature_names[i] for i in range(len(feature_names)) if selected_indices[i]]
    
    print(f"Selected {len(selected_features)} features out of {len(feature_names)}")
    
    # Calculate feature importances for reporting
    selected_importances = [importances[i] for i in range(len(importances)) if selected_indices[i]]
    feature_importance = pd.DataFrame({
        'Feature': selected_features,
        'Importance': selected_importances
    }).sort_values('Importance', ascending=False)
    
    print("Top 10 selected features:")
    print(feature_importance.head(10))
    
    # Save feature selection results
    feature_importance.to_csv(os.path.join(output_dir, 'results', f'selected_features_{method}.csv'), index=False)
    
    # Plot feature importances
    plt.figure(figsize=(14, 10))
    top_features = feature_importance.head(20)
    ax = sns.barplot(x='Importance', y='Feature', data=top_features, palette='viridis')
    ax.set_title(f'Top 20 Selected Features ({method} method)', fontsize=15)
    plt.tight_layout()
    save_figure(plt.gcf(), f'selected_features_{method}.png')
    
    return selector, feature_importance, selected_features

def hyperparameter_tune(X_train, y_train, X_test, y_test, preprocessor, feature_selector=None):
    """Hyperparameter tuning for multiple models with feature selection"""
    print("\nPerforming hyperparameter tuning for all models...")
    
    # Define the models and their hyperparameter spaces
    models = {
        'RandomForest': {
            'model': RandomForestRegressor(random_state=42),
            'param_grid': {
                'model__n_estimators': randint(100, 500),
                'model__max_depth': [None, 10, 20, 30],
                'model__min_samples_split': randint(2, 10),
                'model__min_samples_leaf': randint(1, 5),
                'model__max_features': ['sqrt', 'log2', None]
            }
        },
        'GradientBoosting': {
            'model': GradientBoostingRegressor(random_state=42),
            'param_grid': {
                'model__n_estimators': randint(100, 500),
                'model__learning_rate': uniform(0.01, 0.2),
                'model__max_depth': randint(3, 10),
                'model__min_samples_split': randint(2, 10),
                'model__min_samples_leaf': randint(1, 5),
                'model__subsample': uniform(0.6, 0.4)
            }
        },
        'XGBoost': {
            'model': XGBRegressor.XGBRegressor(random_state=42),
            'param_grid': {
                'model__n_estimators': randint(100, 500),
                'model__learning_rate': uniform(0.01, 0.2),
                'model__max_depth': randint(3, 10),
                'model__min_child_weight': randint(1, 10),
                'model__subsample': uniform(0.6, 0.4),
                'model__colsample_bytree': uniform(0.6, 0.4)
            }
        },
        'ElasticNet': {
            'model': ElasticNet(random_state=42),
            'param_grid': {
                'model__alpha': uniform(0.01, 1),
                'model__l1_ratio': uniform(0, 1)
            }
        }
    }
    
    # Tune each model and store results
    tuned_models = {}
    
    for name, config in models.items():
        print(f"\nTuning {name}...")
        
        # Create pipeline with preprocessing and optional feature selection
        if feature_selector is not None:
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('selector', feature_selector),
                ('model', config['model'])
            ])
        else:
            pipeline = Pipeline([
                ('preprocessor', preprocessor),
                ('model', config['model'])
            ])
        
        # RandomizedSearchCV for efficient tuning
        search = RandomizedSearchCV(
            pipeline,
            param_distributions=config['param_grid'],
            n_iter=20,
            cv=5,
            scoring='neg_root_mean_squared_error',
            n_jobs=-1,
            random_state=42,
            verbose=1
        )
        
        # Fit the search
        search.fit(X_train, y_train)
        
        # Get best model
        best_model = search.best_estimator_
        
        # Evaluate on test set
        y_pred = best_model.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        
        print(f"{name} - Best parameters: {search.best_params_}")
        print(f"{name} - Best CV score: {-search.best_score_:.4f} (RMSE)")
        print(f"{name} - Test RMSE: {rmse:.4f}")
        print(f"{name} - Test R2: {r2:.4f}")
        print(f"{name} - Test MAE: {mae:.4f}")
        
        # Save best parameters
        with open(os.path.join(output_dir, 'results', f'{name}_best_params.txt'), 'w') as f:
            f.write(f"Best parameters for {name}:\n")
            for param, value in search.best_params_.items():
                f.write(f"{param}: {value}\n")
            f.write(f"\nCV score (RMSE): {-search.best_score_:.4f}")
            f.write(f"\nTest RMSE: {rmse:.4f}")
            f.write(f"\nTest R2: {r2:.4f}")
            f.write(f"\nTest MAE: {mae:.4f}")
        
        # Store results
        tuned_models[name] = {
            'model': best_model,
            'best_params': search.best_params_,
            'cv_score': -search.best_score_,
            'test_rmse': rmse,
            'test_r2': r2,
            'test_mae': mae,
            'predictions': y_pred
        }
        
        # Save predictions
        pd.DataFrame({
            'Actual': y_test,
            'Predicted': y_pred,
            'Error': y_test - y_pred,
            'Absolute Error': np.abs(y_test - y_pred)
        }).to_csv(os.path.join(output_dir, 'results', f'{name}_predictions.csv'), index=False)
        
        # Plot actual vs predicted
        plt.figure(figsize=(12, 8))
        plt.scatter(y_test, y_pred, alpha=0.6, color=colors[2])
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.xlabel('Actual', fontsize=14)
        plt.ylabel('Predicted', fontsize=14)
        plt.title(f'{name} (Tuned): Actual vs Predicted', fontsize=16)
        plt.tight_layout()
        save_figure(plt.gcf(), f'{name}_tuned_actual_vs_predicted.png')
        
        # Residual plot
        plt.figure(figsize=(12, 8))
        residuals = y_test - y_pred
        plt.scatter(y_pred, residuals, alpha=0.6, color=colors[3])
        plt.axhline(y=0, color='r', linestyle='-', linewidth=1)
        plt.xlabel('Predicted', fontsize=14)
        plt.ylabel('Residuals', fontsize=14)
        plt.title(f'{name} (Tuned): Residual Plot', fontsize=16)
        plt.tight_layout()
        save_figure(plt.gcf(), f'{name}_tuned_residuals.png')
        
        # Feature importance for tree-based models
        if name in ['RandomForest', 'GradientBoosting', 'XGBoost']:
            try:
                # Access model from pipeline
                model = best_model.named_steps['model']
                
                # Calculate permutation importance - more reliable than built-in feature importances
                perm_importance = permutation_importance(
                    best_model, X_test, y_test, n_repeats=10, random_state=42
                )
                
                # Get feature names after preprocessing
                if hasattr(best_model.named_steps['preprocessor'], 'transformers_'):
                    feature_names = []
                    for name_tr, transformer, cols in best_model.named_steps['preprocessor'].transformers_:
                        if name_tr == 'num':
                            feature_names.extend(cols)
                        elif name_tr == 'cat':
                            # For categorical features, get one-hot encoded feature names
                            ohe = transformer
                            if hasattr(ohe, 'get_feature_names_out'):
                                cat_features = ohe.get_feature_names_out(cols)
                                feature_names.extend(cat_features)
                            else:
                                # Fallback approach
                                for col in cols:
                                    unique_values = X_train[col].dropna().unique()
                                    for val in unique_values:
                                        feature_names.append(f"{col}_{val}")
                    
                    if feature_selector is not None:
                        # Get the selected features indices
                        selected_indices = best_model.named_steps['selector'].get_support()
                        # Filter feature names to only include selected ones
                        if len(feature_names) > len(selected_indices):
                            feature_names = [f for i, f in enumerate(feature_names) if i < len(selected_indices) and selected_indices[i]]
                        else:
                            feature_names = [f for i, f in enumerate(feature_names) if i < len(selected_indices)]
                
                # Adjust feature names and importance to match in length                
                min_len = min(len(feature_names), len(perm_importance.importances_mean))
                feature_names = feature_names[:min_len]
                importances = perm_importance.importances_mean[:min_len]
                
                # Create DataFrame for visualization
                perm_importance_df = pd.DataFrame({
                    'Feature': feature_names,
                    'Importance': importances
                }).sort_values('Importance', ascending=False)
                
                # Save permutation importance
                perm_importance_df.to_csv(os.path.join(output_dir, 'results', f'{name}_permutation_importance.csv'), index=False)
                
                # Plot top 20 features
                plt.figure(figsize=(14, 10))
                top_features = perm_importance_df.head(20)
                ax = sns.barplot(x='Importance', y='Feature', data=top_features, palette='viridis')
                ax.set_title(f'{name}: Top 20 Features by Permutation Importance', fontsize=15)
                plt.tight_layout()
                save_figure(plt.gcf(), f'{name}_permutation_importance.png')
                
                # 3. Add correct partial dependence plots for top features
                top_feature_names = perm_importance_df.head(5)['Feature'].tolist()
                
                for feat_name in top_feature_names:
                    if feat_name in feature_names:
                        feat_idx = feature_names.index(feat_name)
                        
                        try:
                            # Generate partial dependence data
                            pd_values = partial_dependence(
                                best_model, 
                                X_test,
                                features=[feat_idx],
                                kind="average"
                            )
                            
                            # Plot
                            plt.figure(figsize=(10, 6))
                            plt.plot(pd_values["values"][0], pd_values["average"][0], '-o')
                            plt.xlabel(feat_name, fontsize=14)
                            plt.ylabel(f'Partial dependence on {target}', fontsize=14)
                            plt.title(f'Partial Dependence Plot for {feat_name}', fontsize=16)
                            plt.grid(True, alpha=0.3)
                            plt.tight_layout()
                            save_figure(plt.gcf(), f'{name}_pdp_{feat_name}.png')
                        except Exception as e:
                            print(f"Warning: Could not generate partial dependence plot for {feat_name}: {e}")
                            
            except Exception as e:
                print(f"Warning: Could not generate feature importance for {name}: {e}")
    
    return tuned_models

def compare_models(tuned_models):
    """Compare model performance"""
    print("\nComparing model performance...")
    
    # Compile results
    comparison_data = []
    
    for name, results in tuned_models.items():
        comparison_data.append({
            'Model': f"{name} (Tuned)",
            'RMSE': results['test_rmse'],
            'R2': results['test_r2'],
            'MAE': results['test_mae'],
            'CV_RMSE': results['cv_score']
        })
    
    # Create comparison DataFrame
    comparison = pd.DataFrame(comparison_data).sort_values('RMSE')
    
    print("\nModel Comparison:")
    print(comparison)
    
    # Save model comparison
    comparison.to_csv(os.path.join(output_dir, 'results', 'model_comparison.csv'), index=False)
    
    # Plot comparison - RMSE (lower is better)
    plt.figure(figsize=(14, 8))
    ax = sns.barplot(x='Model', y='RMSE', data=comparison, palette='viridis')
    ax.set_title('Model Comparison: RMSE (lower is better)', fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_ylabel('Root Mean Squared Error (RMSE)', fontsize=14)
    
    # Add values on bars
    for i, v in enumerate(comparison['RMSE']):
        ax.text(i, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    save_figure(plt.gcf(), 'model_comparison_rmse.png')
    
    # Plot comparison - R2 (higher is better)
    plt.figure(figsize=(14, 8))
    ax = sns.barplot(x='Model', y='R2', data=comparison, palette='viridis')
    ax.set_title('Model Comparison: R² (higher is better)', fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_ylabel('Coefficient of Determination (R²)', fontsize=14)
    
    # Add values on bars
    for i, v in enumerate(comparison['R2']):
        ax.text(i, v + 0.01, f"{v:.3f}", ha='center', va='bottom', fontsize=12)
    
    plt.tight_layout()
    save_figure(plt.gcf(), 'model_comparison_r2.png')
    
    return comparison

def generate_feature_effect_report(best_model, feature_importance, X_train, y_train, X_test, y_test, numerical_cols, categorical_cols):
    """Generate detailed reports on how features affect the target variable"""
    print("\nGenerating feature effect report...")
    
    # Get top 10 important features
    top_features = feature_importance.head(10)['Feature'].tolist()
    
    # Feature effect report
    report = "# Feature Effect Report\n\n"
    report += "This report details how the top features affect the target variable '% dry after 24h in incubator'.\n\n"
    
    # Add top features section
    report += "## Top 10 Most Important Features\n\n"
    report += feature_importance.head(10).to_markdown(index=False) + "\n\n"
    
    # Add interpretation for each category of feature
    report += "## Feature Interpretations\n\n"
    
    # Group features by type
    feature_types = {}
    for feat in top_features:
        if "_x_" in feat:
            feature_types.setdefault("Interaction", []).append(feat)
        elif "_to_" in feat:
            feature_types.setdefault("Ratio", []).append(feat)
        elif "_squared" in feat or "_sqrt" in feat:
            feature_types.setdefault("Polynomial", []).append(feat)
        elif "_log" in feat:
            feature_types.setdefault("Logarithmic", []).append(feat)
        elif "_missing" in feat:
            feature_types.setdefault("Missing Indicator", []).append(feat)
        elif "_binned" in feat:
            feature_types.setdefault("Binned", []).append(feat)
        elif feat in numerical_cols:
            feature_types.setdefault("Numerical", []).append(feat)
        else:
            feature_types.setdefault("Categorical", []).append(feat)
    
    # Add interpretation for each feature type
    for feat_type, feats in feature_types.items():
        report += f"### {feat_type} Features\n\n"
        report += f"The following {feat_type.lower()} features are important for prediction:\n\n"
        
        for feat in feats:
            report += f"- **{feat}**"
            
            # Add specific interpretation based on feature type
            if feat_type == "Interaction":
                report += f": This represents the combined effect of the constituent features. A high importance indicates that the interaction between these variables significantly affects dryness.\n"
            elif feat_type == "Ratio":
                report += f": This represents the relative proportion between two features. Its importance suggests that the balance between these parameters is crucial for predicting dryness.\n"
            elif feat_type == "Polynomial":
                report += f": This indicates a non-linear relationship with the target. The importance suggests that changes in this feature have an accelerating or diminishing effect on dryness.\n"
            elif feat_type == "Logarithmic":
                report += f": This transformed feature captures the logarithmic relationship with the target, suggesting that proportional changes in the original feature affect dryness.\n"
            elif feat_type == "Missing Indicator":
                report += f": This shows that the presence or absence of data for this feature is predictive of dryness.\n"
            elif feat_type == "Binned":
                report += f": This indicates that specific ranges of this feature have distinct effects on dryness.\n"
            elif feat_type == "Numerical":
                report += f": This original numerical feature directly impacts dryness outcomes.\n"
            elif feat_type == "Categorical":
                report += f": Specific categories of this feature are associated with different dryness outcomes.\n"
            
        report += "\n"
    
    # Add process parameter recommendations
    report += "## Process Parameter Recommendations\n\n"
    report += "Based on the model and feature importance analysis, consider the following recommendations for optimizing the manufacturing process:\n\n"
    
    # Extract key process parameters from top features
    process_parameters = set()
    for feat in top_features:
        # Extract original parameter names from feature expressions
        if "_x_" in feat:
            parts = feat.split("_x_")
            process_parameters.update(parts)
        elif "_to_" in feat:
            parts = feat.split("_to_")
            process_parameters.update(parts)
        elif any(suffix in feat for suffix in ["_squared", "_sqrt", "_log", "_binned", "_missing"]):
            # Extract original parameter
            base_feat = feat.split("_")[0]
            process_parameters.add(base_feat)
        else:
            process_parameters.add(feat)
    
    # Filter out feature expressions
    process_parameters = [p for p in process_parameters if not any(x in p for x in ["_x_", "_to_", "_squared", "_sqrt", "_log", "_binned", "_missing"])]
    
    # Add recommendations for each key process parameter
    for param in process_parameters:
        if param in numerical_cols:
            report += f"- **{param}**: Monitor and control this parameter closely as it significantly affects dryness outcomes.\n"
        elif param in categorical_cols:
            report += f"- **{param}**: Choose optimal categories for this parameter based on desired dryness outcomes.\n"
    
    # Add general recommendations
    report += "\n### General Recommendations:\n\n"
    report += "1. **Focus on Key Parameters**: Prioritize control and monitoring of the top features identified in this analysis.\n"
    report += "2. **Consider Interactions**: Pay attention to how parameters interact with each other, as these interactions are important predictors.\n"
    report += "3. **Maintain Data Quality**: Continue collecting complete data for all parameters to improve future model performance.\n"
    report += "4. **Use the Prediction Model**: Utilize the trained model to predict outcomes before conducting expensive experiments.\n"
    
    # Save the report
    with open(os.path.join(output_dir, 'results', 'feature_effect_report.md'), 'w') as f:
        f.write(report)
    
    return report

def generate_report(df, target, categorical_cols, numerical_cols, engineered_features, 
                   feature_importance, model_comparison, best_model_name, tuned_models,
                   scaling_method, feature_selection_method):
    """Generate a comprehensive report"""
    
    report = f"""
    # Machine Learning Analysis Report: Predicting %dry after 24h in incubator

    ## Executive Summary
    This report presents a comprehensive machine learning approach to predict the "%dry after 24h in incubator" metric, which is a critical quality indicator for the manufacturing process. By analyzing various input parameters and applying advanced feature engineering and modeling techniques, we've developed a model that can reliably predict this metric before conducting expensive experiments.

    ## Dataset Overview
    - Total samples: {df.shape[0]}
    - Features used: {len(categorical_cols) + len(numerical_cols)} original + {len(engineered_features)} engineered
    - Target variable: {target}
    
    ## Data Preprocessing
    
    ### Feature Selection
    - Excluded UV viscosity (cP) due to high multicollinearity with Emulsion viscosity (cP)
    - Excluded high-cardinality categorical features to prevent overfitting
    - Removed metadata columns not relevant for prediction
    - Separated outcome variables from input parameters
    
    ### Missing Value Handling
    - Used customized imputation strategies based on missing data patterns
    - Created missing value indicators for features with high missing rates
    - Applied KNN imputation for complex patterns
    
    ### Outlier Detection
    - Applied IQR method to identify and cap outliers
    - Preserved data distribution while removing extreme values
    
    ## Feature Engineering
    
    ### Physical Relationships
    - Created physically meaningful interaction features
    - Derived process-relevant ratios (e.g., residence time, flow ratios)
    - Generated energy density indicators
    
    ### Mathematical Transformations
    - Applied logarithmic transformations to handle skewed distributions
    - Created polynomial features to capture non-linear relationships
    - Used binning techniques for key parameters
    
    ### Feature Scaling
    - {scaling_method.title()} scaling applied to numerical features
    
    ### Feature Selection
    - {feature_selection_method.title()} method used to identify most important features
    
    ## Most Important Features
    Top 10 features by importance:
    {feature_importance.head(10).to_string(index=False)}
    
    ## Model Performance
    {model_comparison.to_string(index=False)}
    
    ### Best Model: {best_model_name}
    """
    
    if best_model_name.replace(" (Tuned)", "") in tuned_models:
        best_params = tuned_models[best_model_name.replace(" (Tuned)", "")]['best_params']
        report += f"""
    Best hyperparameters:
    {best_params}
    """
    
    report += """
    ## Key Findings
    
    1. Feature engineering significantly improved model performance, especially physically meaningful ratio features
    2. The feature importance analysis revealed which input parameters most strongly influence the dryness outcome
    3. Removing multicollinear features (UV viscosity) improved model robustness without losing predictive power
    4. Process parameters related to viscosity, flow rates, and energy input are critical predictors
    
    ## Engineering Insights
    
    1. Emulsion viscosity strongly influences dryness outcomes
    2. The ratio between core and emulsion viscosity is a key predictor
    3. Flow rates and their ratios significantly impact product quality
    4. Energy input relative to material flow affects curing efficiency
    
    ## Practical Applications
    
    1. Use the model to predict dryness before conducting experiments
    2. Optimize process parameters based on feature importance analysis
    3. Maintain tight control on the most influential parameters
    4. Continue data collection focused on the key parameters identified
    
    ## Recommendations
    
    1. Implement the prediction model as a decision support tool for experiment planning
    2. Design targeted experiments to verify model predictions
    3. Optimize the manufacturing process by focusing on the most impactful parameters
    4. Establish robust monitoring for key process variables
    5. Continue improving the model with new experimental data
    """
    
    # Save report
    with open(os.path.join(output_dir, 'results', 'ml_analysis_report.md'), 'w') as f:
        f.write(report)
    
    print("\nReport generated: ml_analysis_report.md")
    
    return report

def create_prediction_function(best_model, numerical_cols, categorical_cols):
    """Create a function to make predictions on new data"""
    
    def predict_dry_percentage(input_data):
        """
        Predict the %dry after 24h in incubator based on input parameters.
        
        Parameters:
        input_data (dict): Dictionary of input parameters
        
        Returns:
        float: Predicted %dry value
        """
        # Convert input data to DataFrame
        input_df = pd.DataFrame([input_data])
        
        # Ensure all required columns are present
        for col in numerical_cols + categorical_cols:
            if col not in input_df.columns:
                raise ValueError(f"Missing required column: {col}")
        
        # Make prediction
        prediction = best_model.predict(input_df)
        
        return prediction[0]
    
    # Print example usage
    print("\nPrediction Function Created:")
    print("Example usage:")
    print("predict_dry_percentage({")
    for col in numerical_cols[:3]:
        print(f"    '{col}': value,")
    for col in categorical_cols[:2]:
        print(f"    '{col}': 'value',")
    print("    ...\n})")
    
    # Create a sample input template to save
    template = {}
    for col in numerical_cols:
        template[col] = "ENTER_NUMERIC_VALUE"
    for col in categorical_cols:
        template[col] = "ENTER_CATEGORICAL_VALUE"
    
    # Save the template to a file
    with open(os.path.join(output_dir, 'results', 'prediction_template.json'), 'w') as f:
        import json
        json.dump(template, f, indent=4)
    
    return predict_dry_percentage

def save_model_files(best_model, feature_importance, numerical_cols, categorical_cols):
    """Save model files for future use"""
    import joblib
    
    print("\nSaving model files...")
    
    # Save the model
    joblib.dump(best_model, os.path.join(output_dir, 'models', 'best_model.pkl'))
    
    # Save feature lists
    with open(os.path.join(output_dir, 'models', 'model_features.json'), 'w') as f:
        import json
        feature_data = {
            'numerical_columns': numerical_cols,
            'categorical_columns': categorical_cols,
            'important_features': feature_importance['Feature'].tolist()[:20]
        }
        json.dump(feature_data, f, indent=4)
    
    # Create a simple prediction script
    script = """
import pandas as pd
import joblib
import json
import os

# Load the model
model = joblib.load('best_model.pkl')

# Load feature information
with open('model_features.json', 'r') as f:
    features = json.load(f)

numerical_cols = features['numerical_columns']
categorical_cols = features['categorical_columns']

def predict_dry_percentage(input_data):
    \"\"\"
    Predict the %dry after 24h in incubator based on input parameters.
    
    Parameters:
    input_data (dict): Dictionary of input parameters
    
    Returns:
    float: Predicted %dry value
    \"\"\"
    # Convert input data to DataFrame
    input_df = pd.DataFrame([input_data])
    
    # Ensure all required columns are present
    for col in numerical_cols + categorical_cols:
        if col not in input_df.columns:
            raise ValueError(f"Missing required column: {col}")
    
    # Make prediction
    prediction = model.predict(input_df)
    
    return prediction[0]

if __name__ == "__main__":
    # Example usage
    print("\\nDry Percentage Predictor")
    print("-------------------------")
    print("Enter values for the following parameters:")
    
    # Create input dictionary
    input_data = {}
    
    # Get numerical inputs
    for col in numerical_cols:
        while True:
            try:
                value = float(input(f"{col}: "))
                input_data[col] = value
                break
            except ValueError:
                print("Please enter a numeric value.")
    
    # Get categorical inputs
    for col in categorical_cols:
        input_data[col] = input(f"{col}: ")
    
    # Make prediction
    try:
        prediction = predict_dry_percentage(input_data)
        print(f"\\nPredicted %dry after 24h in incubator: {prediction:.4f}")
    except Exception as e:
        print(f"Error making prediction: {e}")
"""
    
    with open(os.path.join(output_dir, 'models', 'predict.py'), 'w') as f:
        f.write(script)
    
    # Create a simple GUI for prediction
    gui_script = """
import pandas as pd
import numpy as np
import joblib
import json
import tkinter as tk
from tkinter import ttk, messagebox
import os

# Load the model
model_path = 'best_model.pkl'
if not os.path.exists(model_path):
    model_path = os.path.join('models', 'best_model.pkl')

model = joblib.load(model_path)

# Load feature information
features_path = 'model_features.json'
if not os.path.exists(features_path):
    features_path = os.path.join('models', 'model_features.json')

with open(features_path, 'r') as f:
    features = json.load(f)

numerical_cols = features['numerical_columns']
categorical_cols = features['categorical_columns']

class PredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Dry Percentage Predictor")
        self.root.geometry("800x600")
        
        # Create main frame
        main_frame = ttk.Frame(root, padding="10")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Create scrollable frame for inputs
        canvas = tk.Canvas(main_frame)
        scrollbar = ttk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        self.scrollable_frame = ttk.Frame(canvas)
        
        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Title label
        title_label = ttk.Label(self.scrollable_frame, 
                               text="Dry Percentage Predictor", 
                               font=("Arial", 16, "bold"))
        title_label.grid(row=0, column=0, columnspan=2, pady=10, sticky="w")
        
        # Description label
        desc_label = ttk.Label(self.scrollable_frame, 
                              text="Enter parameter values to predict % dry after 24h in incubator", 
                              font=("Arial", 10))
        desc_label.grid(row=1, column=0, columnspan=2, pady=5, sticky="w")
        
        # Create input fields
        self.input_vars = {}
        row = 2
        
        # Numerical inputs
        num_label = ttk.Label(self.scrollable_frame, 
                             text="Numerical Parameters", 
                             font=("Arial", 12, "bold"))
        num_label.grid(row=row, column=0, columnspan=2, pady=10, sticky="w")
        row += 1
        
        for col in numerical_cols:
            label = ttk.Label(self.scrollable_frame, text=f"{col}:")
            label.grid(row=row, column=0, sticky="w", padx=5, pady=2)
            
            var = tk.StringVar()
            entry = ttk.Entry(self.scrollable_frame, textvariable=var)
            entry.grid(row=row, column=1, sticky="ew", padx=5, pady=2)
            
            self.input_vars[col] = var
            row += 1
        
        # Categorical inputs
        cat_label = ttk.Label(self.scrollable_frame, 
                             text="Categorical Parameters", 
                             font=("Arial", 12, "bold"))
        cat_label.grid(row=row, column=0, columnspan=2, pady=10, sticky="w")
        row += 1
        
        for col in categorical_cols:
            label = ttk.Label(self.scrollable_frame, text=f"{col}:")
            label.grid(row=row, column=0, sticky="w", padx=5, pady=2)
            
            var = tk.StringVar()
            entry = ttk.Entry(self.scrollable_frame, textvariable=var)
            entry.grid(row=row, column=1, sticky="ew", padx=5, pady=2)
            
            self.input_vars[col] = var
            row += 1
        
        # Predict button
        predict_button = ttk.Button(self.scrollable_frame, 
                                   text="Predict", 
                                   command=self.predict)
        predict_button.grid(row=row, column=0, columnspan=2, pady=20)
        row += 1
        
        # Result frame
        result_frame = ttk.LabelFrame(self.scrollable_frame, text="Prediction Result")
        result_frame.grid(row=row, column=0, columnspan=2, sticky="ew", padx=5, pady=10)
        
        self.result_var = tk.StringVar(value="Results will appear here")
        result_label = ttk.Label(result_frame, textvariable=self.result_var, font=("Arial", 12))
        result_label.pack(padx=10, pady=10)
        
        # Configure grid column weights
        self.scrollable_frame.columnconfigure(1, weight=1)
    
    def predict(self):
        try:
            # Collect input values
            input_data = {}
            
            # Process numerical inputs
            for col in numerical_cols:
                try:
                    value = float(self.input_vars[col].get())
                    input_data[col] = value
                except ValueError:
                    messagebox.showerror("Input Error", f"Please enter a valid number for {col}")
                    return
            
            # Process categorical inputs
            for col in categorical_cols:
                value = self.input_vars[col].get()
                if not value.strip():
                    messagebox.showerror("Input Error", f"Please enter a value for {col}")
                    return
                input_data[col] = value
            
            # Make prediction
            input_df = pd.DataFrame([input_data])
            prediction = model.predict(input_df)[0]
            
            # Update result
            self.result_var.set(f"Predicted %dry after 24h in incubator: {prediction:.4f}")
            
        except Exception as e:
            messagebox.showerror("Prediction Error", f"Error making prediction: {str(e)}")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = PredictionApp(root)
    root.mainloop()
"""
    
    with open(os.path.join(output_dir, 'models', 'prediction_gui.py'), 'w') as f:
        f.write(gui_script)
    
    print("Model files saved to the 'Predictions-Final/models' folder")
    print("- best_model.pkl: The trained model")
    print("- model_features.json: Feature information")
    print("- predict.py: Command-line prediction script")
    print("- prediction_gui.py: Simple GUI application for predictions")

def main():
    # Start timer
    start_time = time.time()
    
    # Step 1: Load and explore data
    df, target = load_and_explore_data(file_path)
    
    # Step 2: Identify relevant columns (exclude metadata, outcome columns, and multicollinear features)
    relevant_cols = identify_relevant_columns(df, target)
    
    # Step 3: Analyze missing value patterns
    missing_patterns = analyze_missing_values(df, relevant_cols)
    
    # Step 4: Analyze features
    categorical_cols = analyze_categorical_features(df, target, relevant_cols)
    numerical_cols = analyze_numerical_features(df, target, relevant_cols)
    
    # Step 5: Filter categorical columns to avoid overfitting
    categorical_cols = filter_categorical_columns(df, categorical_cols, threshold=10)
    
    # Step 6: Feature engineering
    df_engineered, all_numerical_cols, engineered_features = feature_engineering(
        df, numerical_cols, categorical_cols, target)
    
    # Step 7: Create imputation strategy with indicator variables
    numerical_imputers, categorical_imputer, indicator_cols = create_imputation_strategy(
        df_engineered, all_numerical_cols, categorical_cols, missing_patterns)
    
    # Step 8: Prepare data with comprehensive preprocessing
    X_train, X_test, y_train, y_test, numerical_cols_with_indicators = prepare_data(
        df_engineered, target, categorical_cols, all_numerical_cols, 
        numerical_imputers, categorical_imputer, indicator_cols)
    
    # Step 9: Build preprocessing pipeline with scaling options
    scaling_method = 'standard'  # Options: 'standard', 'minmax', 'robust'
    preprocessor = build_preprocessing_pipeline(numerical_cols_with_indicators, categorical_cols, scaling_method)
    
    # Step 10: Feature selection
    feature_selection_method = 'importance'  # Options: 'importance', 'correlation', 'stability'
    feature_selector, feature_importance, selected_features = select_features(
        X_train, y_train, numerical_cols_with_indicators, categorical_cols, 
        preprocessor, method=feature_selection_method)
    
    # Step 11: Hyperparameter tuning for all models with feature selection
    tuned_models = hyperparameter_tune(X_train, y_train, X_test, y_test, preprocessor, feature_selector)
    
    # Step 12: Compare models
    model_comparison = compare_models(tuned_models)
    
    # Step 13: Identify best model
    best_model_name = model_comparison.iloc[0]['Model']
    print(f"\nBest model: {best_model_name}")
    
    # Get the best model
    best_model_name_clean = best_model_name.replace(" (Tuned)", "")
    best_model = tuned_models[best_model_name_clean]['model']
    
    # Step 14: Generate feature effect report
    feature_effect_report = generate_feature_effect_report(
        best_model, feature_importance, X_train, y_train, X_test, y_test,
        numerical_cols_with_indicators, categorical_cols)
    
    # Step 15: Create prediction function
    prediction_function = create_prediction_function(
        best_model, numerical_cols_with_indicators, categorical_cols)
    
    # Step 16: Save model files for future use
    save_model_files(best_model, feature_importance, numerical_cols_with_indicators, categorical_cols)
    
    # Step 17: Generate report
    report = generate_report(
        df, target, categorical_cols, numerical_cols, engineered_features,
        feature_importance, model_comparison, best_model_name, tuned_models,
        scaling_method, feature_selection_method)
    
    # Calculate execution time
    end_time = time.time()
    execution_time = end_time - start_time
    hours, remainder = divmod(execution_time, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    print("\nAnalysis complete!")
    print(f"Total execution time: {int(hours)}h {int(minutes)}m {int(seconds)}s")
    print(f"Best model: {best_model_name}")
    print(f"All results saved to '{output_dir}' folder")
    print("Please check the generated report and figures for detailed analysis.")

if __name__ == "__main__":
    main()

Loading data...

Dataset shape: (202, 33)

First 5 rows:
  Column 1 Date of experiment  \
0  AXF0020         2024-07-24   
1  AXF0021         2024-07-30   
2  AXF0022         2024-08-02   
3  AXF0023         2024-08-09   
4  AXF0024         2024-08-09   

                                   Aims & Hypothesis Core used  \
0                                       Baseline run  BNC00017   
1  Does removing PEDGA from UV remove the 'wickin...  BNC00022   
2  Repeat of AXF0021, at smaller scale to create ...  BNC00017   
3   Size screen as a function of dispersed flow rate  BNC00017   
4   Size screen as a function of dispersed flow rate  BNC00017   

  Core Formulation  Core Viscosity (cP)   UV used UV formulation  \
0     OG (66/27/7)               4560.0  BNU00015          AUF69   
1     OG (66/27/7)               6180.0  BNU00016          AUF85   
2     OG (66/27/7)               6852.0  BNU00017          AUF85   
3     OG (66/27/7)               6756.0  BNU00018          AUF85   
4     O

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>